In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

### 데이터 로드
- 크롤링한 네이버 기사

In [3]:
article_df = pd.read_csv("article.csv")
article_df.tail(2)

,category,content,link,title
598,105,민주노총 전국공공운수노동조합(이하 공공운수노조)이 21일 세종시 어진동 과학기술정보...,https://news.naver.com/main/read.nhn?mode=LSD&...,"공공연구노조 ""출연연 용역근로자 정규직 전환 촉구"" 결의대회"
599,105,"3국 IT국장회의…공개 SW, 저작권자 허락 라이선스 따라 복제·수정·재배포 가능(...",https://news.naver.com/main/read.nhn?mode=LSD&...,"한·중·일, 5G·AI 기술 구현 위해 '공개 SW' 적극 활용 합의"


In [4]:
len(article_df)

600

### 데이터 전처리
- 데이터 셋 분리

In [10]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size=0.1, random_state=1)

In [11]:
len(X_train), len(X_test), len(y_train), len(y_test)

(540, 60, 540, 60)

### 모델 만들기
- 파이프라인으로 만들기

In [12]:
clf = Pipeline([
    ("vect", TfidfVectorizer()),
    ("clf", MultinomialNB(alpha=0.1)),
])

### 학습

In [13]:
model = clf.fit(X_train.values.astype("str"), y_train)

### 모뎅 성능 확인

In [14]:
y_pred = model.predict(X_test)

In [15]:
# 실제 데이터 몇 개와 비교
list(y_test[:5]), list(y_pred[:5])

([104, 104, 105, 104, 102], [104, 104, 105, 104, 102])

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.69      0.90      0.78        10
         101       0.86      0.75      0.80         8
         102       0.80      0.40      0.53        10
         103       0.67      0.57      0.62         7
         104       1.00      0.88      0.93        16
         105       0.60      1.00      0.75         9

   micro avg       0.77      0.77      0.77        60
   macro avg       0.77      0.75      0.74        60
weighted avg       0.80      0.77      0.76        60



### 모델 사용하기
- 새로운 데이터로 분류 예측

In [17]:
categories = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [18]:
# 문장이 길면 정확도가 더 높아진다. 짧으면 떨어진다.
contents = [
    "네이버와 카카오는 드론 기술 발전에 주력",
    "요즘 환율 주가 예측이 불가",
    "광화문 대한애국당 천막, 격렬 저항 속 철거",
]

In [19]:
datas = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주력,105
1,요즘 환율 주가 예측이 불가,101
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",103


In [22]:
# 각 카테고리에 대한 확률을 보여준다.
model.predict_proba([contents[0]])

array([[0.07564372, 0.13719217, 0.14515926, 0.07983495, 0.09079541,
        0.47137449]])

In [23]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(lambda data: round(max(model.predict_proba([data])[0]), 2)) # list로 넣어줘야된다.
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/과학,0.47
1,요즘 환율 주가 예측이 불가,101,경제,0.22
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",103,생활/문화,0.17


### 모델 저장
- pickle 사용하기

In [24]:
# 학습된 객체를 저장
pickle.dump(model, open("clf.pkl", "wb"))